In [41]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pmdarima.arima import auto_arima
from statsmodels.tsa.statespace.sarimax import SARIMAX
from sklearn.metrics import mean_absolute_error, mean_squared_error
from pandas.tseries.offsets import BQuarterEnd
import warnings
warnings.filterwarnings('ignore')

In [42]:
df = pd.read_excel("AIB_History_Training_2014-2020.xlsx", sheet_name='HistoricalTransactions')


In [43]:
df = df.drop(columns=["FileFolder", "FileName", "Report_Title", "Report_RunDate", "Report_AsOfDate", "Currency", "Institution", "Branch", "TransactionBackdateFlag", "TransactionCode", "TransactionGroup"])
df.head()

,Report_TransactionEffectiveDate,TransactionAmount
0,2013-12-30,-972.619715
1,2013-12-30,-75.956876
2,2013-12-30,127.430470
3,2013-12-30,-38.300469
4,2013-12-30,-117.467137


In [44]:
df.set_index("Report_TransactionEffectiveDate", inplace=True)
df.head()

,TransactionAmount
Report_TransactionEffectiveDate,
2013-12-30,-972.619715
2013-12-30,-75.956876
2013-12-30,127.430470
2013-12-30,-38.300469
2013-12-30,-117.467137


In [45]:
df = df.groupby(["Report_TransactionEffectiveDate"])["TransactionAmount"].sum().asfreq('B').fillna(0).to_frame()
df

,TransactionAmount
Report_TransactionEffectiveDate,
2013-12-30,-1076.913729
2013-12-31,-41733.273687
2014-01-01,-3465.890925
2014-01-02,-45506.653405
2014-01-03,-18386.381900
...,...
2020-12-25,1613.384621
2020-12-28,0.000000
2020-12-29,82156.304473


In [46]:
# Check for extreme outliers
q1, q3 = df['TransactionAmount'].quantile([0.25, 0.75])
iqr = q3 - q1
lower_bound = q1 - 1.5 * iqr
upper_bound = q3 + 1.5 * iqr

outliers = df[(df['TransactionAmount'] < lower_bound) | (df['TransactionAmount'] > upper_bound)]
print("Outliers detected:", len(outliers))

Outliers detected: 63


In [47]:
df['TransactionAmount'] = np.clip(df['TransactionAmount'], lower_bound, upper_bound)


In [48]:
def find_best_sarima_params(data, seasonal=True, m=7):
    model = auto_arima(data, seasonal=seasonal, m=m, stepwise=True, suppress_warnings=True, trace=False)
    return model.order, model.seasonal_order

model_order, model_seasonal = find_best_sarima_params(df)


In [49]:
def fit_arima_model(data, order, seasonal):
    model = SARIMAX(data, order=order, seasonal=seasonal, enforce_stationarity=False, enforce_invertibility=False)
    results = model.fit(disp=False)
    return results

model_test = fit_arima_model(df, model_order, model_seasonal)

In [50]:
def evaluate_model(model, actual_data, ):
    predicted = model.fittedvalues
    mae = mean_absolute_error(actual_data, predicted)
    rmse = np.sqrt(mean_squared_error(actual_data, predicted))
    print(f"Model Evaluation: MAE={mae:.2f}, RMSE={rmse:.2f}")

    return mae, rmse

In [51]:
model_mae, model_rmse = evaluate_model(model_test, df)


Model Evaluation: MAE=16190.74, RMSE=21082.92


In [52]:
actual_model = fit_arima_model(df, model_order, model_seasonal)

last_date = df.index[-1]
forecast_days = 90
# Calculate dates for forecasting
future_dates = pd.date_range(start=last_date + pd.Timedelta(days=1), periods=forecast_days, freq='B')

forecast = actual_model.forecast(steps=len(future_dates))
forecasts_df = pd.DataFrame({
    'total_forecast': forecast,  # Total transaction forecast
}, index=future_dates)

forecasts_df

,total_forecast
2021-01-01,26385.013719
2021-01-04,26385.013719
2021-01-05,26385.013719
2021-01-06,26385.013719
2021-01-07,26385.013719
...,...
2021-04-30,26385.013719
2021-05-03,26385.013719
2021-05-04,26385.013719
2021-05-05,26385.013719
